<a href="https://colab.research.google.com/github/sayarghoshroy/Hate-Speech-Detection/blob/master/tweet_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import xlrd
import re
import pickle
import csv

In [2]:
# Uncomment if you're running it for the first time
!pip install ekphrasis
from ekphrasis.classes.segmenter import Segmenter
# to leverage word statistics from Twitter
seg_tw = Segmenter(corpus = "twitter")

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple
Reading twitter - 1grams ...
Reading twitter - 2grams ...


/home/tathagata.raha/miniconda3/envs/fastai/lib/python3.8/site-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [3]:
# Uncomment if you're running it for the first time
!pip install tweet-preprocessor
import preprocessor as tweet_proc

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [4]:
# Uncomment if you're running it for the first time
!pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


#### *Raw Datasets are hosted [here](https://drive.google.com/drive/folders/1TuHRQQ41lK9oXJhlhsRiMjMnczuKn_kF?usp=sharing).*

In [5]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

In [123]:
def make_list(proc_obj):
  if proc_obj == None:
    return []
  
  store = []
  for unit in proc_obj:
    store.append(unit.match)
  
  return store

def emotext(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",", "").replace(":", "").split()))
    return text

In [124]:
# For 2020 Datasets

is_hindi = 1

# For Train Data
# datatype = "train"
# For English
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/english.xlsx"

# For Hindi
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/hindi.xlsx"
# is_hindi = 1

# For German
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/german.xlsx"

# For Test Data
datatype = "train"
# For English
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_test_data/english_test_1509.csv"

# For Hindi
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_test_data/hindi_test_1509.csv"
# is_hindi = 1

# For German
file_name = "data/valid.tsv"

In [125]:
# Initializing Lists
datapoints_count = 0
see_index = True

tweets = []
raw_tweet_texts = []
tokenized_tweets = []
hashtags = []
smileys = []
emojis = []
urls = []
mentions = []
numbers = []
reserveds = []
clean = []
task_1_labels = []
task_2_labels = []
tweet_ids = []
hasoc_ID = []

In [126]:
def strip_list(listie):
  stripped = []
  for item in listie:
    stripped.append(item.strip())
  return stripped

def hindi_clean(line, parse_obj):
  # beta
  tokens = line.replace(":", " : ").replace(",", " , ").replace(";", " ; ").split(" ")
  valid_stri = ""

  for raw_token in tokens:
    token = raw_token.strip()
    if token in strip_list(make_list(parse_obj.hashtags)):
      continue
    if token in strip_list(make_list(parse_obj.smileys)):
      continue
    if token in strip_list(make_list(parse_obj.emojis)):
      continue
    if token in strip_list(make_list(parse_obj.urls)):
      continue
    if token in strip_list(make_list(parse_obj.mentions)):
      continue
    if token in strip_list(make_list(parse_obj.numbers)):
      continue
    if token in strip_list(make_list(parse_obj.reserved)):
      continue
    valid_stri = valid_stri + " " + token
  return valid_stri.strip()

In [127]:
# pd.read_csv(file_name, delimiter = "\t", quoting=csv.QUOTE_NONE, error_bad_lines=False).iloc[150:200]

In [128]:
if datatype == 'train' or 'test':
#     workbook = xlrd.open_workbook(file_name)
#     sheet = workbook.sheet_by_index(0)

#     for row in range(sheet.nrows):
#         line = sheet.row_values(row)

    file = open(file_name, 'r')
    count = 0
    file_reader = csv.reader(file, delimiter = "\t", quoting=csv.QUOTE_NONE)
    first = 1
    for line in file_reader:
        if first:
            first = 0
            continue
#         if see_index == True:
#             see_index = False
#             continue

        datapoints_count += 1
        tweet_ids.append(line[0])
        if datatype == 'train':
            task_1_labels.append(line[2])
        tweets.append(line[1].replace("\n", " "))
        temp = line[1].replace("\n", " ")
        parse_obj = tweet_proc.parse(line[1].replace("\n", " "))
        tokenized_tweets.append(tweet_proc.tokenize(line[1].replace("\n", " ")))
        tag = strip_list(make_list(parse_obj.hashtags))
        hashtags.append(tag)
        smiley = strip_list(make_list(parse_obj.smileys))
        smileys.append(smiley)
        emoji = strip_list(make_list(parse_obj.emojis))
        emojis.append(emoji)
        url = strip_list(make_list(parse_obj.urls))
        urls.append(url)
        mention = strip_list(make_list(parse_obj.mentions))
        mentions.append(mention)
        numbers.append(strip_list(make_list(parse_obj.numbers)))
        reserveds.append(strip_list(make_list(parse_obj.reserved)))
        for i in url + mention + smiley + emoji + tag:
#             print(i, tem)
            temp = temp.replace(i, " ")
        clean.append(temp)
        if is_hindi == 0:
          raw_tweet_texts.append(tweet_proc.clean(line[1].replace("\n", " ")))
        else:
          raw_tweet_texts.append(hindi_clean(line[1].replace("\n", " "), parse_obj))

    print("Number of Datapoints: " + str(datapoints_count))

Number of Datapoints: 811


In [129]:
datapoints_count

811

In [130]:
# if datatype == 'test':
#     file = open(file_name, 'r')
#     file_reader = csv.reader(file, delimiter = "\t")
#     for line in file_reader:
# #         if see_index == True:
# #             see_index = False
# #             continue
# #         print(line)
#         datapoints_count += 1
#         tweet_ids.append(line[0])
# #         task_1_labels.append(line[2])
# #         task_2_labels.append(line[3])
# #         hasoc_ID.append(line[4])
#         tweets.append(line[1].replace("\n", " "))

#         parse_obj = tweet_proc.parse(line[1].replace("\n", " "))
#         tokenized_tweets.append(tweet_proc.tokenize(line[1].replace("\n", " ")))
#         hashtags.append(strip_list(make_list(parse_obj.hashtags)))
#         smileys.append(strip_list(make_list(parse_obj.smileys)))
#         emojis.append(strip_list(make_list(parse_obj.emojis)))
#         urls.append(strip_list(make_list(parse_obj.urls)))
#         mentions.append(strip_list(make_list(parse_obj.mentions)))
#         numbers.append(strip_list(make_list(parse_obj.numbers)))
#         reserveds.append(strip_list(make_list(parse_obj.reserved)))

#         if is_hindi == 0:
#           raw_tweet_texts.append(tweet_proc.clean(line[1].replace("\n", " ")))
#         else:
#           raw_tweet_texts.append(hindi_clean(line[1].replace("\n", " "), parse_obj))

#     print("Number of Datapoints: " + str(datapoints_count))

In [131]:
# Viewing Created Dataset

print("Tweets:")
print(tweets[0: 5])

print("Raw Texts:")
print(raw_tweet_texts[0: 5])

print("Hashtags:")
print(hashtags[0: 5])

print("Smileys:")
print(smileys[0: 5])

print("Emojis:")
print(emojis[0: 5])

print("Urls:")
print(urls[0: 5])

print("Mentions:")
print(mentions[0: 5])

print("Numbers:")
print(numbers[0: 5])

print("Reserved Words:")
print(reserveds[0: 5])
print("Cleaned")
print(clean[0:5])
print("Task Labels:")
print(task_1_labels[0: 5])
# print(task_2_labels[0: 5])

Tweets:
['दृढ़ इच्छा शक्ति से परिपूर्ण प्रणबदा के लिए देशहित सर्वोच्च रहा।  उनका निधन हम सब के लिए अपूरणीय क्षति है। ईश्वर दिवंगत आत्मा को अपने श्रीचरणों में स्थान दें। शोक संतप्त परिजनों के प्रति संवेदनाएं। ऊं शांति!!!', 'भारतीय जनता पार्टी rss वाले इतने गिरे हुए हैं जहां मैं रहती हूं वहां मेरी जासूसी  करा रहें है उसकी जासूस की पहचान मुझे अच्छी तरह है rss बीजेपी वाले की जासूस दिल्ली में कौन है उत्तर प्रदेश में कौन है हरियाणा राजस्थान में कौन है सबकी पहचान है मुझे मेरी नजर से बच नहीं सकते हो', 'कोरोना से निपटने की तैयारी / दिल्ली में 10 हजार बेड वाला दुनिया का सबसे बड़ा कोविड केयर सेंटर शुरू, राजनाथ-शाह ने डीआरडीओ के 1 हजार बेड वाले सेंटर का भी उद्घाटन किया https://t.co/9rlQowAsFh #Delhi @ArvindKejriwal  @rajnathsingh @AmitShah @DRDO_India @WHO @crpfindia @ITBP_official', 'गवर्नर कॉन्फ्रेंस में PM मोदी बोले- शिक्षा नीति में सरकार का दखल कम होना चाहिए https://t.co/ZvKgxk6dbd', 'यूपी: गाजीपुर में Toilet घोटाला, प्रधान व सचिव ने किया लाखों का गबन, मुर्दों के नाम पर बनवा डाले शौचालय  #UP h

In [132]:
# Generating Emoji Texts
emoji_texts = []

for emo_list in emojis:
  texts = []
  for emoji in emo_list:
    text = emotext(emoji)
    texts.append(text.replace("_", " "))
  emoji_texts.append(texts)

print("Emoji Descriptions:")
print(emoji_texts[0: 5])

Emoji Descriptions:
[[], [], [], [], []]


In [133]:
# Segmenting Hashtags
segmented_hashtags = []

for hashset in hashtags:
  segmented_set = []
  for tag in hashset:
    word = tag[1: ]
    # removing the hash symbol
    segmented_set.append(seg_tw.segment(word))
  segmented_hashtags.append(segmented_set)

print("Segmented Hashtags:")
print(segmented_hashtags[0: 5])

Segmented Hashtags:
[[], [], ['delhi'], [], ['up']]


In [134]:
name = 'valid.pickle'
dickie = {}
dickie['tweet_id'] = tweet_ids
if datatype == 'train':
    dickie['task_1'] = task_1_labels
dickie['full_tweet'] = tweets
dickie['tweet_raw_text'] = raw_tweet_texts
dickie['hashtags'] = hashtags
dickie['smiley'] = smileys
dickie['emoji'] = emojis
dickie['url'] = urls
dickie['mentions'] = mentions
dickie['numerals'] = numbers
dickie['reserved_word'] = reserveds
dickie['emotext'] = emoji_texts
dickie['segmented_hash'] = segmented_hashtags
# dickie['clean'] = clean
with open(name, 'wb') as f:
  pickle.dump(dickie, f)

In [135]:
with open(name, 'rb') as f:
  try_dict = pickle.load(f)

sizes = []
for key in try_dict.keys():
  sizes.append(len(try_dict[key]))

# Verifying if all sizes are equal
print(sizes)

[811, 811, 811, 811, 811, 811, 811, 811, 811, 811, 811, 811, 811]


In [136]:
try_dict.keys()

dict_keys(['tweet_id', 'task_1', 'full_tweet', 'tweet_raw_text', 'hashtags', 'smiley', 'emoji', 'url', 'mentions', 'numerals', 'reserved_word', 'emotext', 'segmented_hash'])